In [1]:
import sys
import time

import cv2
import dlib
from IPython.display import Image
import numpy as np

from ImageHelpers import bgr2png
from VideoStreams import stream_factory

ModuleNotFoundError: No module named 'PIL'

In [ ]:
STREAM_URL = 'http://192.168.8.219:8080/'
CLASSES = ["background",
           "aeroplane",
           "bicycle",
           "bird",
           "boat",
           "bottle",
           "bus",
           "car",
           "cat",
           "chair",
           "cow",
           "diningtable",
           "dog",
           "horse",
           "motorbike",
           "person",
           "pottedpolant",
           "sheep",
           "sofa",
           "train",
           "tvmonitor"]
SEEKING = ["car", "motorbike"]
PROTOTEXT = "mobilenet_ssd/MobileNetSSD_deploy.prototxt"
MODEL = "mobilenet_ssd/MobileNetSSD_deploy.caffemodel"
MIN_CONFIDENCE = 0.5

In [ ]:
# initializing pipeline components
stream = stream_factory(STREAM_URL)
model = cv2.dnn.readNetFromCaffe(PROTOTEXT, MODEL)
trackers = []

In [ ]:
# process each frame
for (timestamp, frame) in stream:
    if frame is None:
        continue

    sys.stdout.write('Run inference on frame... ')
    start = time.time()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    (img_height, img_width) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (img_width, img_height), 127.5)
    model.setInput(blob)
    detections = model.forward()
    stop = time.time()
    sys.stdout.write(f'Done { stop - start }')
    
    break

In [ ]:
for i in np.arange(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    label = CLASSES[int(detections[0, 0, i, 1])]
    if confidence < MIN_CONFIDENCE:
        continue
    if label not in SEEKING:
        continue
        
    box = detections[0, 0, i, 3:7] * \
        np.array([img_width, img_height, img_width, img_height])
    box = box.astype("int")

    tracker = dlib.correlation_tracker()
    rect = dlib.rectangle(*box)
    tracker.start_track(rgb, rect)
    trackers.append(tracker)

    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
    cv2.putText(frame, label, (box[0], box[1] - 15),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

In [ ]:
Image(data=bgr2png(frame)) 